In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import transforms
import torch.optim as optim
from torch import nn
from tqdm.notebook import tqdm
from torchsummary import summary
from torchvision import datasets
from torchvision import transforms


import utils
from utils import softclip
from models.cnn_sigmaVAE import CNN_sigmaVAE
from models.cnn_sigmacVAE import CNN_sigmacVAE
from models.cnn_sigmaVAE_flow import CNN_sigmaVAE_flow
from models.cnn_sigmacVAE_flow import CNN_sigmacVAE_flow
from trainer import Trainer

import evaluation_utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [2]:
window_size=32
cond_window_size=13
batch_size=256
latent_dim=10
X_train_data, X_test_data, X_train_tensor, cond_train_tensor, X_test_tensor, cond_test_tensor, df_Y_test, trainloader, valloader, testloader = utils.read_machine_data_cvae_with_validation('../../datasets/ServerMachineDataset/machine-1-1', window_size, cond_window_size, batch_size, .3)
 


In [3]:
num_feats = X_train_data.shape[-1]

model = CNN_sigmacVAE(latent_dim, window_size, cond_window_size=cond_window_size, num_feats=num_feats).to(device)
model.cuda() if torch.cuda.is_available() else model.cpu()

trainer = Trainer(data_name = '1-1', model_type = 'cvae', flow_type='', early_stop_patience=200)
model, flag = trainer.train_model(model, num_epochs=10, learning_rate=.005, trainloader=trainloader, valloader=valloader)


In [4]:
utils.plot_reconstruction(model, model_type='cvae',dataloader=trainloader)
utils.plot_reconstruction(model, model_type='cvae',dataloader=testloader)

MSE : 0.0124093988
MSE : 0.0211777524


In [4]:


model = CNN_sigmacVAE(latent_dim, window_size, cond_window_size=cond_window_size, num_feats=38, use_probabilistic_decoder=True).to(device)
model.cuda() if torch.cuda.is_available() else model.cpu()

trainer = Trainer(data_name = '1-1', model_type = 'cvae', flow_type='', early_stop_patience=200)
model, flag = trainer.train_model(model, num_epochs=1000, learning_rate=.005, trainloader=trainloader, valloader=valloader)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "C:\Python37\lib\site-packages\IPython\core\interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-c926c6917ff3>", line 5, in <module>
    model, flag = trainer.train_model(model, num_epochs=1000, learning_rate=.005, trainloader=trainloader, valloader=valloader)
  File "c:\Users\raimi\Desktop\school\vaeflows\2d-flows\trainer.py", line 73, in train_model
    loss.backward()
  File "C:\Python37\lib\site-packages\torch\tensor.py", line 195, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "C:\Python37\lib\site-packages\torch\autograd\__init__.py", line 99, in backward
    allow_unreachable=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call

KeyboardInterrupt: 

In [ ]:
utils.plot_reconstruction_prob_decoder(model, 'cvae', trainloader, X_train_tensor)

utils.plot_reconstruction_prob_decoder(model, 'cvae', trainloader, X_test_tensor)